In [ ]:
# Measuring TF sub-metric a) and b), N=7

# function: measure similarity submetric1 : lines removed then added
def calculate_levenshtein_similarity_event1(hunk1,hunk2):
    removed_lines_added = 0  
    for idx, val in enumerate(hunk1):
        for index, value in enumerate(hunk2):
            if val.startswith('+++') == False and val.startswith('---') == False and value.startswith('+++') == False and value.startswith('---') == False:                   
                if val.startswith('-') == True and value.startswith('+') == True:
                    # measure the similarity between two strings
                    sim = Levenshtein.ratio(val.lstrip('-'), value.lstrip('+'))
                    if sim > 0.8:
                        removed_lines_added = 1
                        return removed_lines_added
    return removed_lines_added


# function: measure similarity submetric2 :  lines added then removed
def calculate_levenshtein_similarity_event2(hunk1,hunk2):
    added_lines_removed = 0 
    for idx, val in enumerate(hunk1):
        for index, value in enumerate(hunk2):
            if val.startswith('+++') == False and val.startswith('---') == False and value.startswith('+++') == False and value.startswith('---') == False:
                if val.startswith('+') == True and value.startswith('-') == True:
                    sim = Levenshtein.ratio(val.lstrip('+'), value.lstrip('-'))
                    if sim > 0.8:
                        added_lines_removed = 1
                        return added_lines_removed                    
    return added_lines_removed


# handle different hunk numbers
def handle_multi_hunks(f_commit,s_commit,count_hunks_f,count_hunks_s):
    removed_lines_added = 0
    added_lines_removed = 0
    
    # 1.1 both files have only one modified code area/hunk
    if len(count_hunks_f) == 1 and len(count_hunks_s) == 1:         
        hunk_content_1 = f_commit[count_hunks_f[0]]
        line_num_1 = re.split('[-,+,@@]',hunk_content_1)
        star_line_1 = 0
        if line_num_1[5].isdigit() and line_num_1[3].isdigit():
            star_line_1 = int(line_num_1[5])
            if (int(line_num_1[3]) < int(line_num_1[5])):
                star_line_1 = int(line_num_1[3])
            
        a1 = int(line_num_1[3]) + int(line_num_1[4])
        b1 = 0
        if line_num_1[5].isdigit() and line_num_1[6].isdigit():
            b1 = int(line_num_1[5]) + int(line_num_1[6])
        end_line_1 = b1
        if a1 > b1:
            end_line_1 = a1
            
        hunk_content_2 = s_commit[count_hunks_s[0]]
        line_num_2 = re.split('[-,+,@@]',hunk_content_2)
        star_line_2 = 0
        if line_num_2[3].isdigit() and line_num_2[5].isdigit():
            star_line_2 = int(line_num_2[5])      
            if (int(line_num_2[3]) < int(line_num_2[5])):
                star_line_2 = int(line_num_2[3])
                
        a2 = int(line_num_2[3]) + int(line_num_2[4])
        b2 = 0
        if line_num_2[5].isdigit() and line_num_2[6].isdigit():
            b2 = int(line_num_2[5]) + int(line_num_2[6])
        end_line_2 = b2
        if a2 > b2:
            end_line_2 = a2
             
        if (int(star_line_2) <= int(star_line_1) < int(end_line_2)) or (int(star_line_2) < int(end_line_1) <= int(end_line_2)):
            e= calculate_levenshtein_similarity_event1(f_commit,s_commit)
            f = calculate_levenshtein_similarity_event2(f_commit,s_commit)
            removed_lines_added = removed_lines_added + e
            added_lines_removed = added_lines_removed + f 

            
    # 1.2 one file has only one modified code area, another file has more than one modified code areas
    if (len(count_hunks_f) == 1 or len(count_hunks_s) == 1) and (len(count_hunks_f) + len(count_hunks_s) > 2):
        iter_hunk_list = []
        target_hunk_list = []
        target_commit_list = []
        iter_commit_hunk_list = []
        
        if len(count_hunks_f) == 1:
            iter_hunk_list = count_hunks_s
            target_hunk_list = count_hunks_f
            target_commit_list = f_commit
            iter_commit_hunk_list = s_commit

        if len(count_hunks_s) == 1:
            iter_hunk_list = count_hunks_f
            target_hunk_list = count_hunks_s
            target_commit_list = s_commit
            iter_commit_hunk_list = f_commit

        target_hunk_content = target_commit_list[target_hunk_list[0]]
        target_line_num = re.split('[-,+,@@]',target_hunk_content)
        target_star_line = int(target_line_num[5])
        
        if (int(target_line_num[3]) < int(target_line_num[5])):
            target_star_line = int(target_line_num[3])
        a = int(target_line_num[3]) + int(target_line_num[4])
        b = 0
        if target_line_num[5].isdigit() and target_line_num[6].isdigit():
            b = int(target_line_num[5]) + int(target_line_num[6])
        target_end_line = b
        if a > b:
            target_end_line = a
        # ['', '', '', '347', '7', '350', '21', '', 'defplot_grid(']
        different_hunk_segment = []
        for i in range(len(iter_hunk_list)):
            cont_for_each_hunk = ''
            if 0 <= i < len(iter_hunk_list)-1:
                cont_for_each_hunk =  iter_commit_hunk_list[iter_hunk_list[i]:iter_hunk_list[i+1]]  
            if i == len(iter_hunk_list)-1:
                cont_for_each_hunk =  iter_commit_hunk_list[iter_hunk_list[i]:]
            different_hunk_segment.append(cont_for_each_hunk)

        # check if there is a same modified code area/hunk
        for nn, each_hunk in enumerate(different_hunk_segment):
            # each hunk header
            cur_hunk_head = ''
            for xx, each_line in enumerate(each_hunk):
                if each_line.startswith('@@'):
                    cur_hunk_head = each_line

            iter_line_num = re.split('[-,+,@@]',cur_hunk_head)
            cur_star_line = iter_line_num[5]
            if (int(iter_line_num[3]) < int(iter_line_num[5])):
                cur_star_line = int(target_line_num[3])
            a = int(iter_line_num[3]) + int(iter_line_num[4])
            b = int(iter_line_num[5]) + int(iter_line_num[6])
            cur_end_line = b
            if a > b:
                cur_end_line = a
            if (int(target_star_line) <= int(cur_star_line) < int(target_end_line)) or (int(target_star_line) < int(cur_end_line) <= int(target_end_line)):
                # same area
                if target_commit_list == f_commit:
                    e = calculate_levenshtein_similarity_event1(target_commit_list,each_hunk)
                    f = calculate_levenshtein_similarity_event2(target_commit_list,each_hunk)
                    removed_lines_added = removed_lines_added + e
                    added_lines_removed = added_lines_removed + f 
                else:
                    e = calculate_levenshtein_similarity_event1(each_hunk,target_commit_list)
                    f = calculate_levenshtein_similarity_event2(each_hunk,target_commit_list)
                    removed_lines_added = removed_lines_added + e
                    added_lines_removed = added_lines_removed + f 

                    
    # 1.3 both two files have more than one modified code areas
    if len(count_hunks_f) > 1 and len(count_hunks_s) > 1:
        different_hunk_segment1 = []
        different_hunk_segment2 = []
        for i in range(len(count_hunks_f)):
            # each hunk section
            cont_for_each_hunk = ''
            if 0 <= i < len(count_hunks_f)-1:
                cont_for_each_hunk =  f_commit[count_hunks_f[i]:count_hunks_f[i+1]]  
            if i == len(count_hunks_f)-1:
                cont_for_each_hunk =  f_commit[count_hunks_f[i]:]
            different_hunk_segment1.append(cont_for_each_hunk)
            
        for i in range(len(count_hunks_s)):
            # each hunk section
            cont_for_each_hunk = ''
            if 0 <= i < len(count_hunks_s)-1:
                cont_for_each_hunk = s_commit[count_hunks_s[i]:count_hunks_s[i+1]]  
            if i == len(count_hunks_s)-1:
                cont_for_each_hunk =  s_commit[count_hunks_s[i]:]
            different_hunk_segment2.append(cont_for_each_hunk)

        for aa, each_hunk1 in enumerate(different_hunk_segment1):
            # each hunk header
            cur_hunk_head1 = ''
            for ee, each_line in enumerate(each_hunk1):
                if each_line.startswith('@@'):
                    cur_hunk_head1 = each_line

            iter_line_num1 = re.split('[-,+,@@]',cur_hunk_head1)            
            cur_star_line1 = -100
            a1 = 0
            b1 = 0
            if len(iter_line_num1) >= 6 and iter_line_num1[5].isdigit():
                if iter_line_num1[3].isdigit() and iter_line_num1[4].isdigit() and iter_line_num1[6].isdigit():
                     
                    cur_star_line1 = int(iter_line_num1[5])
                    if (int(float(iter_line_num1[3])) < int(iter_line_num1[5])):
                        cur_star_line1 = int(float(iter_line_num1[3]))

                    a1 = int(float(iter_line_num1[3])) + int(iter_line_num1[4])      
                    b1 = int(iter_line_num1[5]) + int(iter_line_num1[6])
            else:
                continue                
            cur_end_line1 = b1            
            if a1 > b1:
                cur_end_line1 = a1

            for bb, each_hunk2 in enumerate(different_hunk_segment2):
                cur_hunk_head2 = ''
                for cc, each_line in enumerate(each_hunk2):
                    if each_line.startswith('@@'):
                        cur_hunk_head2 = each_line
                iter_line_num2 = re.split('[-,+,@@]',cur_hunk_head2)                
                cur_star_line2 = 0
                a2 = 0
                b2 = 0
                if len(iter_line_num2) >= 6 and iter_line_num2[5].isdigit():
                    if iter_line_num2[3].isdigit() and iter_line_num2[4].isdigit() and iter_line_num2[6].isdigit():
                        
                        cur_star_line2 = iter_line_num2[5]                        
                        if (int(iter_line_num2[3]) < int(iter_line_num2[5])):
                            cur_star_line2 = int(iter_line_num2[3])                    
                        a2 = int(iter_line_num2[3]) + int(iter_line_num2[4])
                        b2 = int(iter_line_num2[5]) + int(iter_line_num2[6])
                else:
                    continue
                    
                cur_end_line2 = b2
                if a2 > b2:
                    cur_end_line2 = a2
                if (int(cur_star_line2) <= int(cur_star_line1) < int(cur_end_line2)) or (int(cur_star_line2) < int(cur_end_line1) <= int(cur_end_line2)):
                    # same area
                    e = calculate_levenshtein_similarity_event1(each_hunk1,each_hunk2)
                    f = calculate_levenshtein_similarity_event2(each_hunk1,each_hunk2)
                    removed_lines_added = removed_lines_added + e
                    added_lines_removed = added_lines_removed + f                     
    return removed_lines_added, added_lines_removed


# function: compare seven successive commits: 1-2-3-4-5-6-7，2-3-4-5-6-7-8
def compare_adjacent_commit_list(my_list,projectID):
    c_removed_lines_added = 0
    c_added_lines_removed = 0
    count_hunks_f = []
    count_hunks_s = []    
    store_old_file_loc_plus = []
    store_old_file_loc_minus = []    
    store_new_file_loc_plus = []
    store_new_file_loc_minus = []
    
    #Total_commit
    n = len(my_list)  
    window_size_commit_list = []
    count =0
    for i in range(len(my_list)-7):
        window_size_commit_list = my_list[i:i+7]         
        # [[1, 2], [1, 3],[1,4],[1,5],[1,6],[1,7]]
        all_compare_pairs = []
        every_compare_pair = []
        for i in range(len(window_size_commit_list)):
            if i+1 < len(window_size_commit_list):
                every_compare_pair.append(window_size_commit_list[0])
                every_compare_pair.append(window_size_commit_list[i+1])
                all_compare_pairs.append(every_compare_pair)
            every_compare_pair = []

        for loc, each_commit_in_pair in enumerate(all_compare_pairs):
            count += 1
            count_hunks_f.clear()
            count_hunks_s.clear()     
            store_old_file_loc_plus.clear()
            store_old_file_loc_minus.clear()
            
            store_new_file_loc_plus.clear()
            store_new_file_loc_minus.clear()

            for idx, val in enumerate(each_commit_in_pair[0]):
                # @@
                if val.startswith('@@') == True:
                    count_hunks_f.append(idx)
                #+++
                if val.startswith('+++') == True:
                    store_old_file_loc_plus.append(idx)
                #---
                if val.startswith('---') == True:
                    store_old_file_loc_minus.append(idx)

            for index, value in enumerate(each_commit_in_pair[1]):
                # @@
                if value.startswith('@@') == True:
                    count_hunks_s.append(index)
                #+++
                if value.startswith('+++') == True:
                    store_new_file_loc_plus.append(index)
                #---
                if value.startswith('---') == True:
                    store_new_file_loc_minus.append(index)                     

            c1 = 0
            c2 = 0
            # case 1:  both commits change only one file
            if len(store_old_file_loc_plus) == 1 and len(store_new_file_loc_plus) == 1:
                # changing the same file
                old_version_file = (each_commit_in_pair[0][store_old_file_loc_plus[0]])
                new_version_file = (each_commit_in_pair[1][store_new_file_loc_plus[0]])

                if old_version_file == new_version_file:
                    c1, c2 = handle_multi_hunks(each_commit_in_pair[0],each_commit_in_pair[1],count_hunks_f,count_hunks_s)


            # case 2:  one commit changes only one file, another commit changes more than one files
            if (len(store_old_file_loc_plus) == 1 or len(store_new_file_loc_plus) == 1) and (len(store_old_file_loc_plus) + len(store_new_file_loc_plus) > 2):
                iter_file_list = []
                target_file_list = []
                target_commit_list = []
                iter_commit_files_list = []

                if len(store_old_file_loc_plus) == 1:
                    iter_file_list = store_new_file_loc_plus
                    target_file_list = store_old_file_loc_plus
                    target_commit_list = each_commit_in_pair[0]
                    iter_commit_files_list = each_commit_in_pair[1]

                if len(store_new_file_loc_plus) == 1:
                    iter_file_list = store_old_file_loc_plus
                    target_file_list = store_new_file_loc_plus
                    target_commit_list = each_commit_in_pair[1]
                    iter_commit_files_list = each_commit_in_pair[0]

                target_file_name = target_commit_list[target_file_list[0]]
                all_iter_filenames = []
                
                # iterate the commit that contains multiple modified files and save different files into a list
                different_file_segment = []
                for i in range(len(iter_file_list)):
                    all_iter_filenames.append(iter_commit_files_list[iter_file_list[i]])
                    list_for_each_file = ''
                    if 0 <= i < len(iter_file_list)-1:                        
                        list_for_each_file =  iter_commit_files_list[iter_file_list[i]:iter_file_list[i+1]-1] 
                    if i == len(iter_file_list)-1:
                        list_for_each_file =  iter_commit_files_list[iter_file_list[i]:]  

                    different_file_segment.append(list_for_each_file)

                # two commits had modifying a same file
                if target_file_name in all_iter_filenames:
                    for i, v in enumerate(different_file_segment):
                        hunk_num_of_cur_file = []
                        current_file_name = ''
                        for jj, kk in enumerate(v):
                            if kk.startswith('@@') == True:
                                hunk_num_of_cur_file.append(jj)
                            if kk.startswith('+++') == True:
                                current_file_name = kk

                        if target_file_name == current_file_name:
                            if target_commit_list == each_commit_in_pair[0]:
                                c1, c2 = handle_multi_hunks(each_commit_in_pair[0],v,count_hunks_f,hunk_num_of_cur_file)
                            else:
                                c1, c2 = handle_multi_hunks(v,each_commit_in_pair[1],hunk_num_of_cur_file,count_hunks_s)


            # case 3: both commits had changed more than one files
            if len(store_old_file_loc_plus) > 1 and len(store_new_file_loc_plus) > 1:
                different_file_segment_c1 = []
                different_file_segment_c2 = []
                
                for i in range(len(store_old_file_loc_plus)):
                    if 0 <= i < len(store_old_file_loc_plus)-1:
                        each_file_diff1 =  each_commit_in_pair[0][store_old_file_loc_plus[i]:store_old_file_loc_plus[i+1]-1]        
                    if i == len(store_old_file_loc_plus)-1:
                        each_file_diff1 =  each_commit_in_pair[0][store_old_file_loc_plus[i]:]        
                    different_file_segment_c1.append(each_file_diff1)

                for i in range(len(store_new_file_loc_plus)):
                    if 0 <= i < len(store_new_file_loc_plus)-1:
                        each_file_diff2 =  each_commit_in_pair[1][store_new_file_loc_plus[i]:store_new_file_loc_plus[i+1]-1]        
                    if i == len(store_new_file_loc_plus)-1:
                        each_file_diff2 =  each_commit_in_pair[1][store_new_file_loc_plus[i]:]        

                    different_file_segment_c2.append(each_file_diff2)
                list_1 = []
                list_2 = []
                for i, each_file1 in enumerate(different_file_segment_c1):
                    hunk_num_of_cur_file1 = []         
                    current_file1 = ''
                    current_file2 = ''
                    for n, e_line in enumerate(each_file1):
                        if e_line.startswith('@@') == True:
                            hunk_num_of_cur_file1.append(n)
                        if e_line.startswith('+++') == True:
                            current_file1 = e_line
                            
                    for a, each_file2 in enumerate(different_file_segment_c2):
                        hunk_num_of_cur_file2 = []
                        for j, e_line2 in enumerate(each_file2):
                            if e_line2.startswith('@@') == True:
                                hunk_num_of_cur_file2.append(j)
                            if e_line2.startswith('+++') == True:
                                current_file2 = e_line2
                        x1 = 0
                        x2 = 0
                        if current_file1 == current_file2:
                            x1, x2 = handle_multi_hunks(each_file1,each_file2,hunk_num_of_cur_file1,hunk_num_of_cur_file2)                     
                            list_1.append(x1)
                            list_2.append(x2)                
                c1 = sum(list_1)
                c2 = sum(list_2)
            c_removed_lines_added = c_removed_lines_added + c1
            c_added_lines_removed = c_added_lines_removed + c2
                                  
    print(projectID,' ',len(my_list),'            ',c_removed_lines_added,'               ',c_added_lines_removed)


import re  
import Levenshtein
# match Date line
p = "(^Date:)(\s*)(Mon|Thu|Tue|Wed|Fri|Sat|Sun)(.+)" 
# the list storing the diff file name for each project, for example three projects below
diff_list = ['4001','4008','4009']
print('ID','TotalCommit','Case1_removed_then_added','Case2_added_then_removed')

for gg, each_diff_file in enumerate(diff_list):    
    diffs_file = path_to_diff_files + each_diff_file +'.txt'    
    f = open(diffs_file,'r',encoding='utf-8',errors='ignore')
    diffs_list = f.readlines() 

    # convert txt to [[C1],[C2],[]...]
    store_date_line_index = []
    for index, val in enumerate(diffs_list):       
        if type(val) == bytes:    
            val = val.decode(errors='ignore')
        
        match_date = re.match(p,val,flags=0) 
        if match_date != None: 
            store_date_line_index.append(index)

    list_after_split = []
    for i in range(len(store_date_line_index)):
        sub_list_for_each_commit='C_'+str(i+1)
        locals()['C_'+str(i+1)]=i 
        # the first commit
        if i == 0:
            sub_list_for_each_commit = diffs_list[:store_date_line_index[1]]
        # the last commit
        if i == (len(store_date_line_index)-1):
            sub_list_for_each_commit = diffs_list[store_date_line_index[i]:] 

        if 0 < i < len(store_date_line_index)-1:
            sub_list_for_each_commit =  diffs_list[store_date_line_index[i]:store_date_line_index[i+1]]
        list_after_split.append(sub_list_for_each_commit) 

    compare_adjacent_commit_list(list_after_split,each_diff_file)